# DeepCAD-RT training pipeline            
<img src="https://github.com/STAR-811/DeepCAD-RT-old/blob/main/images/logo-new.png?raw=true" width = "650" height = "180" align=right />

This file will demonstrate the basic pipeline for training DeepCAD-RT. A TIFF file will be downloaded automatically to be the example data. More information about the method and relevant results can be found in the companion paper：

**Real-time denoising of fluorescence time-lapse imaging enables high-sensitivity observations of biological dynamics beyond the shot-noise limit. bioRxiv (2022).**

In [1]:
from deepcad.train_collection import training_class
from deepcad.movie_display import display
from deepcad.utils import get_first_filename,download_demo

## Select file(s) to be processed (download if not present)
The `download_demo` function will download a demo file and return the full path of it. This demo file will be stored in `datasets`. If you want to use your own data for training, please create a new folder in `datasets` and copy your data into it. 
Then, just change `datasets_path` into the name of your dataset folder. All TIFF files inside the dataset folder will be used for training.

In [2]:
download_demo_file = False
if download_demo_file:
    file_name='fish_localbrain' # select the demo file for training (e.g. 'ATP_3D', 'fish_localbrain', 'NP_3D', ...)
    datasets_path, _ =download_demo(download_filename=file_name)
else:
    datasets_path = 'datasets/simulate_-2.51dBSNR_1000frames_demo'  # folder containing files for training

## Set the parameters for training
Default setting shows the parameters for the demo file, which are also appropriate for most data. You can change these parameters accroding to your data and device. To visualize the training process, you can set the flags `visualize_images_per_epoch` and `save_test_images_per_epoch` according to your demands.

In [3]:
n_epochs = 5                # number of training epochs
GPU = '0'                   # the index of GPU you will use (e.g. '0', '0,1', '0,1,2')
train_datasets_size = 6000  # datasets size for training (how many 3D patches)
patch_xy = 150              # the width and height of 3D patches
patch_t = 150               # the time dimension (frames) of 3D patches
overlap_factor = 0.4        # the overlap factor between two adjacent patches
pth_dir = './pth'           # the path for pth file and result images 
num_workers = 0             # if you use Windows system, set this to 0.

# Setup some parameters for result visualization during training period (optional)
visualize_images_per_epoch = True  # whether to show result images after each epoch
save_test_images_per_epoch = True  # whether to save result images after each epoch

##  Show the input low-SNR data  (optional)
Play an input video (optional). This will load the video into memory and it is not an indispensable step. The OpenCV library was used for display.

In [4]:
display_images = True

if display_images:
    display_filename = get_first_filename(datasets_path)
    print('\033[1;31mDisplaying the first raw file -----> \033[0m')
    print(display_filename)
    display_length = 300     # how many frames to display
    # normalize the image and display
    display(display_filename, display_length=display_length, norm_min_percent=1, norm_max_percent=98)

Displaying the first raw file -----> 
datasets/simulate_-2.51dBSNR_1000frames_demo/simulate_-2.51dBSNR_1000frames.tif


## Create a training object
This will creat a training object by passing all parameters as a dictionary. Parameters not specified in the dictionary will use their default values.

In [5]:
train_dict = {
    # dataset dependent parameters
    'patch_x': patch_xy,                          # the width of 3D patches
    'patch_y': patch_xy,                          # the height of 3D patches
    'patch_t': patch_t,                           # the time dimension (frames) of 3D patches
    'overlap_factor':overlap_factor,              # overlap factor
    'scale_factor': 1,                            # the factor for image intensity scaling
    'select_img_num': 1000000,                    # select the number of frames used for training (use all frames by default)
    'train_datasets_size': train_datasets_size,   # datasets size for training (how many 3D patches)
    'datasets_path': datasets_path,               # folder containing files for training
    'pth_dir': pth_dir,                           # the path for pth file and result images 
    
    # network related parameters
    'n_epochs': n_epochs,                         # the number of training epochs
    'lr': 0.00005,                                # learning rate
    'b1': 0.5,                                    # Adam: bata1
    'b2': 0.999,                                  # Adam: bata2
    'fmap': 16,                                   # model complexity
    'GPU': GPU,                                   # GPU index
    'num_workers': num_workers,                   # if you use Windows system, set this to 0.
    'visualize_images_per_epoch': visualize_images_per_epoch,   # whether to show result images after each epoch
    'save_test_images_per_epoch': save_test_images_per_epoch    # whether to save result images after each epoch
}

tc = training_class(train_dict)

Training parameters -----> 
{'overlap_factor': 0.4, 'datasets_path': 'datasets/simulate_-2.51dBSNR_1000frames_demo', 'n_epochs': 5, 'fmap': 16, 'output_dir': './results', 'pth_dir': './pth', 'batch_size': 1, 'patch_t': 150, 'patch_x': 150, 'patch_y': 150, 'gap_y': 90, 'gap_x': 90, 'gap_t': 90, 'lr': 5e-05, 'b1': 0.5, 'b2': 0.999, 'GPU': '0', 'ngpu': 1, 'num_workers': 0, 'scale_factor': 1, 'train_datasets_size': 200, 'select_img_num': 1000000, 'test_datasize': 400, 'visualize_images_per_epoch': True, 'save_test_images_per_epoch': True}


## Start the training process

Here we lanuch the training precess. The model of each epoch will be saved in the `pth` folder.

In [6]:
tc.run()

Image list for training -----> 
Total stack number ----->  1
Noise image name ----->  simulate_-2.51dBSNR_1000frames.tif
Noise images shape ----->  (1000, 490, 490)
Using 1 GPU(s) for training -----> 
[Epoch 1/5] [Batch 208/208] [Total loss: 133879.38, L1 Loss: 431.95, L2 Loss: 267326.81] [ETA: 0:08:59] [Time cost: 134 s]      
 Testing model of epoch 1 on the first noisy file ----->
Testing image name ----->  simulate_-2.51dBSNR_1000frames.tif
Noise images shape ----->  (400, 490, 490)
 [Patch 100/100] [Time Cost: 15 s] [ETA: 0 s]      
 Displaying the first denoised file ----->

[Epoch 2/5] [Batch 208/208] [Total loss: 146795.33, L1 Loss: 447.59, L2 Loss: 293143.06] [ETA: 0:06:48] [Time cost: 307 s]      
 Testing model of epoch 2 on the first noisy file ----->
Testing image name ----->  simulate_-2.51dBSNR_1000frames.tif
Noise images shape ----->  (400, 490, 490)
 [Patch 100/100] [Time Cost: 14 s] [ETA: 0 s]      
 Displaying the first denoised file ----->

[Epoch 3/5] [Batch 208/20